In [ ]:
import pandas as pd

# load the csv
df = pd.read_csv("../data/Assessment 2 - MMM Weekly.csv")

# show first few rows
print(df.head())

# check columns
print(df.columns)

# check missing values
print(df.isna().sum())

In [ ]:
import matplotlib.pyplot as plt

# Make sure week is datetime and sorted
df['week'] = pd.to_datetime(df['week'])
df = df.sort_values('week')

# Plot revenue over time
plt.figure(figsize=(10,4))
plt.plot(df['week'], df['revenue'], marker='o')
plt.title("Revenue over Time")
plt.xlabel("Week")
plt.ylabel("Revenue")
plt.grid(True)
plt.show()


In [ ]:
# Pick spend columns
spend_cols = ['facebook_spend', 'google_spend', 'tiktok_spend', 
              'instagram_spend', 'snapchat_spend']

# Plot each spend channel vs revenue
import matplotlib.pyplot as plt

plt.figure(figsize=(12,8))

for i, col in enumerate(spend_cols, 1):
    plt.subplot(len(spend_cols), 1, i)
    plt.plot(df['week'], df[col], label=col, color='tab:blue')
    plt.plot(df['week'], df['revenue'], label='revenue', color='tab:orange', alpha=0.7)
    plt.legend(loc='upper right')
    plt.ylabel(col)
    if i == 1:
        plt.title("Spend channels vs Revenue over time")

plt.tight_layout()
plt.show()


In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(12,8), sharex=True)

axs[0].plot(df['week'], df['promotions'], label='promotions', color='purple')
axs[0].plot(df['week'], df['revenue'], label='revenue', color='orange', alpha=0.7)
axs[0].legend(loc='upper right')

axs[1].plot(df['week'], df['emails_send'], label='emails', color='green')
axs[1].plot(df['week'], df['revenue'], label='revenue', color='orange', alpha=0.7)
axs[1].legend(loc='upper right')

axs[2].plot(df['week'], df['sms_send'], label='sms', color='red')
axs[2].plot(df['week'], df['revenue'], label='revenue', color='orange', alpha=0.7)
axs[2].legend(loc='upper right')

axs[3].plot(df['week'], df['average_price'], label='average_price', color='blue')
axs[3].plot(df['week'], df['revenue'], label='revenue', color='orange', alpha=0.7)
axs[3].legend(loc='upper right')

plt.suptitle("Other Marketing Levers vs Revenue")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
import numpy as np

# Features and target
X = df[['facebook_spend','google_spend','tiktok_spend',
        'instagram_spend','snapchat_spend',
        'emails_send','sms_send','social_followers',
        'average_price','promotions']]

y = df['revenue']

# Train-test split (time-based: first 80% train, last 20% test)
split = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

# Fit model
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Metrics
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test RMSE: {rmse:.2f}")
print(f"Test R²: {r2:.2f}")

# Compare actual vs predicted visually
import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.plot(y_test.index, y_test.values, label="Actual")
plt.plot(y_test.index, y_pred, label="Predicted")
plt.title("Baseline Linear Regression: Revenue (Test Set)")
plt.legend()
plt.show()



In [ ]:
import numpy as np

def adstock(series, half_life=2):
    """
    Simple exponential adstock transform.
    half_life = weeks until effect halves.
    """
    decay = 0.5 ** (1/half_life)
    res = np.zeros(len(series))
    carry = 0
    for i, x in enumerate(series):
        carry = x + decay * carry
        res[i] = carry
    return res

# Apply adstock to each spend channel
spend_cols = ['facebook_spend','google_spend','tiktok_spend',
              'instagram_spend','snapchat_spend']

for col in spend_cols:
    df[col + "_adstock"] = adstock(df[col].values, half_life=2)

# Check first few rows
df[[col+"_adstock" for col in spend_cols]].head()


In [ ]:
# Apply log transform to adstocked spends
for col in spend_cols:
    ad_col = col + "_adstock"
    sat_col = col + "_sat"
    df[sat_col] = np.log1p(df[ad_col])   # log1p = log(1+x), safe for zero

# Check new saturation columns
df[[col+"_sat" for col in spend_cols]].head()


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score

# Features: saturated spends + controls
X = df[[col+"_sat" for col in spend_cols] + 
       ['emails_send','sms_send','social_followers','average_price','promotions']]

y = df['revenue']

# Train-test split (time-based: first 80% train, last 20% test)
split = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

# Fit model
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Metrics
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test RMSE: {rmse:.2f}")
print(f"Test R²: {r2:.2f}")

# Plot actual vs predicted
import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.plot(y_test.index, y_test.values, label="Actual")
plt.plot(y_test.index, y_pred, label="Predicted")
plt.title("Improved Linear Regression: Revenue (with Adstock + Saturation)")
plt.legend()
plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression

# Social channels (use the same ones you had before with adstock/saturation if available)
social_features = ['facebook_spend_sat', 'tiktok_spend_sat', 'snapchat_spend_sat', 'instagram_spend_sat']

# Controls (non-media factors)
controls = ['emails_send','sms_send','social_followers','average_price','promotions']

# Stage 1: Google ~ social + controls
X_stage1 = df[social_features + controls]
y_stage1 = df['google_spend']

model_stage1 = LinearRegression()
model_stage1.fit(X_stage1, y_stage1)

# Print coefficients
coef_stage1 = pd.Series(model_stage1.coef_, index=X_stage1.columns)
print("Stage1 coefficients (Google ~ Social + Controls):")
print(coef_stage1.sort_values(ascending=False).round(4))


In [ ]:
# Stage 1 predictions: predicted Google spend
df['google_hat'] = model_stage1.predict(X_stage1)

# Stage 2: Revenue ~ social + predicted Google + controls
X_stage2 = df[social_features + ['google_hat'] + controls]
y_stage2 = df['revenue']

model_stage2 = LinearRegression()
model_stage2.fit(X_stage2, y_stage2)

# Print coefficients
coef_stage2 = pd.Series(model_stage2.coef_, index=X_stage2.columns)
print("Stage2 coefficients (Revenue ~ Social + Predicted Google + Controls):")
print(coef_stage2.sort_values(ascending=False).round(4))


In [ ]:
# Indirect effect for social i = (coef_stage1[social_i]) * (coef_stage2['google_hat'])
indirect_multiplier = coef_stage2['google_hat']

effects = []
for s in social_features:
    direct = coef_stage2.get(s, 0.0)
    indirect = coef_stage1.get(s, 0.0) * indirect_multiplier
    total = direct + indirect
    pct_indirect = (indirect / total) if total != 0 else np.nan
    effects.append({
        "channel": s,
        "direct": direct,
        "indirect": indirect,
        "total": total,
        "pct_indirect": pct_indirect
    })

effects_df = pd.DataFrame(effects)
print("Direct vs Indirect vs Total effects:")
print(effects_df.round(2))


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

def rolling_origin_eval(X, y, n_splits=5, initial_train_size=None):
    n = len(X)
    if initial_train_size is None:
        initial_train_size = int(n * 0.5)
    fold_size = int((n - initial_train_size) / n_splits)
    results = []
    for i in range(n_splits):
        train_end = initial_train_size + i * fold_size
        test_start = train_end
        test_end = min(train_end + fold_size, n)
        if test_start >= test_end:
            continue
        model = Ridge(alpha=1.0)
        model.fit(X.iloc[:train_end], y.iloc[:train_end])
        y_pred = model.predict(X.iloc[test_start:test_end])
        rmse = mean_squared_error(y.iloc[test_start:test_end], y_pred) ** 0.5
        r2 = r2_score(y.iloc[test_start:test_end], y_pred)
        results.append({"fold": i+1, "rmse": rmse, "r2": r2})
    return results

X_stage2 = df[social_features + ['google_hat'] + controls]
y_stage2 = df['revenue']

cv_results = rolling_origin_eval(X_stage2, y_stage2, n_splits=5, initial_train_size=int(len(df)*0.5))
print("Rolling CV results:")
for r in cv_results:
    print(f" Fold {r['fold']}: RMSE={r['rmse']:.2f}, R²={r['r2']:.3f}")


In [ ]:
# Refit on first 80% for diagnostics
split = int(len(df) * 0.8)
model = Ridge(alpha=1.0)
model.fit(X_stage2.iloc[:split], y_stage2.iloc[:split])
y_pred = model.predict(X_stage2.iloc[split:])
resid = y_stage2.iloc[split:] - y_pred

import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

# Residuals vs predictions
plt.figure(figsize=(8,4))
plt.scatter(y_pred, resid)
plt.axhline(0, color='k', linestyle='--')
plt.xlabel("Predicted revenue")
plt.ylabel("Residuals")
plt.title("Residuals vs Predictions (holdout)")
plt.show()

# Autocorrelation of residuals
plot_acf(resid, lags=20)
plt.show()


In [ ]:
# Add lagged revenue features
df['revenue_lag1'] = df['revenue'].shift(1)
df['revenue_lag2'] = df['revenue'].shift(2)

# Drop first 2 rows (they have NaNs from shifting)
df_lagged = df.dropna().copy()

# Update features to include lagged revenue
X_stage2_lag = df_lagged[social_features + ['google_hat'] + controls + ['revenue_lag1','revenue_lag2']]
y_stage2_lag = df_lagged['revenue']

# Refit model
model_lag = Ridge(alpha=1.0)
split = int(len(df_lagged) * 0.8)
model_lag.fit(X_stage2_lag.iloc[:split], y_stage2_lag.iloc[:split])
y_pred_lag = model_lag.predict(X_stage2_lag.iloc[split:])
resid_lag = y_stage2_lag.iloc[split:] - y_pred_lag

print("Test RMSE with lagged revenue:",
      mean_squared_error(y_stage2_lag.iloc[split:], y_pred_lag) ** 0.5)
print("Test R² with lagged revenue:",
      r2_score(y_stage2_lag.iloc[split:], y_pred_lag))

# Residuals vs predictions (after adding lags)
plt.figure(figsize=(8,4))
plt.scatter(y_pred_lag, resid_lag)
plt.axhline(0, color='k', linestyle='--')
plt.xlabel("Predicted revenue")
plt.ylabel("Residuals")
plt.title("Residuals vs Predictions (with lagged revenue)")
plt.show()

# Autocorrelation of residuals
plot_acf(resid_lag, lags=20)
plt.show()


In [ ]:
from xgboost import XGBRegressor

# Use the same features as Stage2 (without lagged revenue, since it hurt)
X_gb = df[social_features + ['google_hat'] + controls]
y_gb = df['revenue']

# Time-based split
split = int(len(df) * 0.8)
X_train, X_test = X_gb.iloc[:split], X_gb.iloc[split:]
y_train, y_test = y_gb.iloc[:split], y_gb.iloc[split:]

# Fit Gradient Boosting
gb_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
gb_model.fit(X_train, y_train)

# Predictions
y_pred_gb = gb_model.predict(X_test)

# Metrics
rmse_gb = mean_squared_error(y_test, y_pred_gb) ** 0.5
r2_gb = r2_score(y_test, y_pred_gb)

print(f"XGBoost Test RMSE: {rmse_gb:.2f}")
print(f"XGBoost Test R²: {r2_gb:.3f}")

# Plot actual vs predicted
plt.figure(figsize=(10,4))
plt.plot(y_test.values, label="Actual")
plt.plot(y_pred_gb, label="Predicted")
plt.title("XGBoost: Revenue (Test Set)")
plt.legend()
plt.show()


In [ ]:

# Final XGBoost Model Training & SHAP Interpretation

# 1. Import necessary libraries
import shap
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import pandas as pd

# 2. Define features and target
social_features = ['facebook_spend_sat', 'tiktok_spend_sat', 'snapchat_spend_sat', 'instagram_spend_sat']
controls = ['emails_send','sms_send','social_followers','average_price','promotions']
X_gb = df[social_features + ['google_hat'] + controls]
y_gb = df['revenue']

# 3. Split the data
split = int(len(df) * 0.8)
X_train, X_test = X_gb.iloc[:split], X_gb.iloc[split:]
y_train, y_test = y_gb.iloc[:split], y_gb.iloc[split:]

# 4. Define and train the XGBoost model
gb_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    enable_categorical=True
)
gb_model.fit(X_train, y_train)

# 5. Create a summary of the training data for the explainer's background dataset
X_train_summary = shap.kmeans(X_train, 10)

# 6. Create the KernelExplainer using a lambda function
# This decouples the explainer from the main model object, avoiding the error
prediction_function = lambda x: gb_model.predict(x)
explainer = shap.KernelExplainer(prediction_function, X_train_summary)

# 7. Calculate SHAP values for the test set
# This will be slower, so please be patient
print("Calculating SHAP values with KernelExplainer... (this may take a moment)")
shap_values = explainer.shap_values(X_test)
print("...done.")

# 8. Generate and display SHAP plots
print("\nSHAP Feature Importance (Bar Plot):")
shap.summary_plot(shap_values, X_test, plot_type="bar")
plt.show()

print("\nSHAP Detailed Summary Plot:")
shap.summary_plot(shap_values, X_test)
plt.show()